<img src="images/banner.png" alt="Folded outcrop" />

# Duplex example
## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from LoopStructural import GeologicalModel
from LoopStructural.visualisation.model_visualisation import LavaVuModelViewer
from LoopStructural.datasets import load_duplex
import matplotlib.pyplot as plt
%matplotlib inline

## Define model bounding box

In [2]:
data, boundary_points = load_duplex()#np.zeros((2,3))


In [3]:
splay_region = lambda pos : np.logical_or(pos[:,0] < (-6.5-model.origin[0])/model.scale_factor, pos[:,0] 
                                          > (5-model.origin[0])/model.scale_factor)

In [4]:
model = GeologicalModel(boundary_points[0,:],boundary_points[1,:])
model.set_model_data(data)

In [5]:
fault1 = model.create_and_add_fault('fault2',
                                    -2.,
                                    nelements=4000,
                                    solver='lu',
                                    interpolatortype='FDI',
                                   damp = True,
#                                     precon=precon,
                                    buffer=np.array([0.5,0.2,0.3]),
                                    regularisation=[2.,.2,.2],
                                    gxxgy=4.,
                                    gxxgz=4.,
                                    gyxgz=4.,
#                                                                         region=interpolation_region
                                    steps=10,
                                    cpw=10
                                    
                                   )
# splay_region = lambda pos : np.logical_or(fault1['feature'].apply_to_points(pos)[:,0] < (-4.5-model.origin[0])/model.scale_factor, 
#                                           fault1['feature'].apply_to_points(pos)[:,0] > (4.5-model.origin[0])/model.scale_factor)
fault2 = model.create_and_add_fault('fault1',
                                    -2,
                                    nelements=4000,
                                    solver='lu',
                                    interpolatortype='FDI',
                                   damp = True,
                                    splayregion=splay_region,
                                    splay = fault1['feature'],
                                    regularisation=[2.,.5,.5],
                                    gxxgy=4.,
                                    gxxgz=4.,
                                    gyxgz=4.,
#                                     precon=precon,
                                    buffer=np.array([0.6,0.2,0.2]),
#                                                                         region=interpolation_region
                                    steps=10,
                                    cpw=10,
                                    overprinted=['fault2'],
                                   )
strati = model.create_and_add_foliation('strati',
                                        solver='lu',
                                        interpolatortype='FDI', 
                                        nelements=1500,
                                        damp=True,
                                        buffer=np.array([0.8,0.4,0.4])
                                       )

c:\users\lgrose\.conda\envs\loop\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
c:\users\lgrose\.conda\envs\loop\lib\site-packages\LoopStructural\modelling\fault\fault_segment.py:176: RuntimeWarning: invalid value encountered in greater
  d[gx > 0] = 1.
c:\users\lgrose\.conda\envs\loop\lib\site-packages\LoopStructural\modelling\fault\fault_segment.py:177: RuntimeWarning: invalid value encountered in less
  d[gx < 0] = 0.
c:\users\lgrose\.conda\envs\loop\lib\site-packages\LoopStructural\modelling\fault\fault_segment.py:200: RuntimeWarning: invalid value encountered in greater
  d[gx > 0] = 1.
c:\users\lgrose\.conda\envs\loop\lib\site-packages\LoopStructural\modelling\fault\fault_segme

In [6]:
viewer = LavaVuModelViewer(model)
viewer.nsteps = np.array([100,20,100])
# viewer.add_vector_field(strati['feature'])#,locations=nodes2)#,model.regular_grid((25,25,12)))

viewer.add_scalar_field(strati['feature'],cmap='tab20')
viewer.lv.rotatex(-90)
viewer.lv['xmin'] = -1
viewer.lv['ymin'] = -1
viewer.lv['zmin'] = -1
viewer.lv['xmax'] = 1
viewer.lv['ymax'] = 1
viewer.lv['zmax'] = 1
viewer.lv['border'] = 0
viewer.lv.image('fault_duplex_scalar_field.png')
viewer.lv.display()